# Package

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait,Select
from selenium.webdriver.support import expected_conditions as EC  #selenium에서 사용할 모듈 import
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

import time
import requests
import re
import csv
import os
import pandas as pd
import  urllib

# Paris baguette

In [ ]:
def GetURL(url):
    driver = webdriver.Chrome("C:\\data\\chromedriver_win32\\chromedriver.exe")  #selenium 사용에 필요한 chromedriver.exe 파일 경로 지정
    driver.get(url) 
    
    time.sleep(3)
    
    return driver

def PageParser(driver):
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    return soup

def CountingDropDown(driver, option_num):
    soup = PageParser(driver)
    SelectArea = soup.find_all('div', class_ = "selectric-hide-select")
    AreaList = SelectArea[option_num].select('option')
    AreaCnt = len(AreaList)
    
    return AreaList, AreaCnt

def SendDown(driver):
    swapToActive = driver.switch_to.active_element
    swapToActive.send_keys(Keys.DOWN)
    swapToActive.send_keys(Keys.ENTER)
    
    time.sleep(5)
    
def SelectSearchMode(driver):
    search = driver.find_element_by_xpath('//*[@id="main"]/div[1]/div[1]/div[1]/ul/li[2]/a')
    search.click()
    

In [ ]:
StoreDict= {'region':[], 'brand' : [], 'branch' :[], 'addr' :[]}
url = 'https://www.paris.co.kr/store/'

driver = GetURL(url)

SelectSearchMode(driver)

Area1List, CntDD1 = CountingDropDown(driver,0)

for i in tqdm(range(CntDD1 - 1), desc = '지역별 정보 수집중.... '):
    try:
        #Area1 선택
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#main > div.store-map.show-timetable > div.store-control > div.store-control-header > div > form > div.location > div.selector.area1 > div"))).click()
        SendDown(driver)
        
        driver.switch_to.active_element

        soup2 = PageParser(driver)
        StoreArea = soup2.find("div", class_ = "store-list")
        StoreBranchs = StoreArea.find_all('span', class_ = 'label')
        StoreAddrs = StoreArea.find_all('p', class_ = 'store-addr')

        for br, addr in zip(StoreBranchs, StoreAddrs):
            StoreDict['region'].append(Area1List[i+1].text)
            StoreDict['brand'].append('파리바게뜨')
            StoreDict['branch'].append(br.text)
            StoreDict['addr'].append(addr.text)

        time.sleep(5)
    
    except:
        print(f"Error : Area1 {Area1List[i+1].text}")
        continue
        
driver.quit()

print("Done")

# Dunkin

In [ ]:
def click_city(cityvalue):
    city = Select(driver.find_element_by_xpath('//*[@id="nform"]/fieldset/p[1]/span[1]/select'))
    city.select_by_index(str(cityvalue))
    
def click_gugun(gugunvalue):
    gugun =Select(driver.find_element_by_xpath('//*[@id="nform"]/fieldset/p[1]/span[2]/select'))
    gugun.select_by_index(str(gugunvalue))

def click_btn():
    btn = driver.find_element_by_xpath('//*[@id="nform"]/fieldset/div[2]/button')
    btn.click()
    
def find_tag(tag,class_):
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    lst = soup.find(tag, class_= class_)
    
    return lst

In [ ]:
url = 'https://www.dunkindonuts.co.kr/store/map.php'

driver = webdriver.Chrome("C:\\data\\chromedriver_win32\\chromedriver.exe")  #selenium 사용에 필요한 chromedriver.exe 파일 경로 지정
driver.get(url) 

time.sleep(3) #화면 표시 기다리기

city_num = 17
gugun_num = 40
tmp_name = []
tmp_addr = []

for i in tqdm(range(1,city_num+1)):
    click_city(i)
    time.sleep(3)

    for k in range(1,gugun_num+1):
        try:
            click_gugun(k)
            click_btn()
            
            time.sleep(5)
        
            lst = find_tag('div','list')
            names = lst.find_all('h3',class_="name")
            addrs = lst.find_all('span')

            for name,addr in zip(names,addrs[::2]):
                try:
                    re_name = re.sub('[\n\t]+','',name.text)
                    re_name = re.sub('^\d+','',re_name)
                    tmp_name.append(re_name)
                    tmp_addr.append(addr.text)
                except:
                    continue
        except:
            continue

driver.quit()

print("Done")

# Baskin robbins

In [ ]:
url = 'http://www.baskinrobbins.co.kr/store/map.php'

driver = webdriver.Chrome("C:\\data\\chromedriver_win32\\chromedriver.exe")  #selenium 사용에 필요한 chromedriver.exe 파일 경로 지정
driver.get(url) 

time.sleep(3) #화면 표시 기다리기

city_num = 17
gugun_num = 40
tmp_name = []
tmp_addr = []

for c_num in tqdm(range(1,city_num+1)):
    click_city(c_num)
    time.sleep(3)

    for g_num in range(1,gugun_num+1):
        try:
            click_gugun(g_num)
            click_btn()
            
            time.sleep(5)
            
            lst = find_tag('div','list')
            names = lst.find_all('h3',class_="name")
            addrs = lst.find_all("address",class_="address")

            for name,addr in zip(names,addrs):
                try:
                    re_name = re.sub('[\n\t]+','',name.text)
                    re_name = re.sub('^\d+','',re_name)+'점'
                    tmp_name.append(re_name)
                    tmp_addr.append(re.sub('주소|[\n\t]+','',addr.text))
                except:
                    tmp_name.append(f'{c_num},{g_num}')
                    tmp_addr.append(f'{c_num},{g_num}')
        except:
            continue

driver.quit()

print("Done")

# Naver maps

In [ ]:
def naver_map_search(keyword, category):
    map_addr = "https://m.map.naver.com/search2/search.naver?query=" + keyword

    driver = webdriver.Chrome("C:\\data\\chromedriver_win32\\chromedriver.exe")  #selenium 사용에 필요한 chromedriver.exe 파일 경로 지정
    driver.get(map_addr) #네이버 모바일 지도 

    try:
        element = WebDriverWait(driver, 10).until(
           EC.presence_of_element_located((By.CLASS_NAME, "Nbox_text"))
        ) #입력창이 뜰 때까지 대기
    finally:
        pass

    time.sleep(3) #화면 표시 기다리기

    # 스크롤 끝까지 내리기
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:   # 0에서부터 영상 맨 마지막까지 긁음.

        # 마우스 스크롤을 끝까지 내리겠음
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        time.sleep(3.0) 

        # 내린 스크롤까지의 높이를 구해서 변수에 넣음
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height: # 지금의 높이가 웹페이지의 끝이라면
            break # 종료

        last_page_height = new_page_height # 그렇지 않으면 계속 스크롤을 진행


    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    title = soup.find_all("div", class_ = "item_tit _title")
    address = soup.find_all("a", class_ = "item_address _btnAddress")

    temp_dict = {'brand': [], 
                 'name' : [], 
                 'branch': [], 
                 'addr' : []
                }

    for tit,addr in zip(title,address):
        brand = keyword
        name = tit.text.split(' ')[0]
        branch = re.sub(category,'',tit.text[tit.text.find(' ')+1:])
        tmpaddr = addr.text[re.search('\s+', addr.text).end():]

        temp_dict['brand'].append(brand)  #brand
        temp_dict['name'].append(name)  #name
        temp_dict['branch'].append(branch)  #branch
        temp_dict['addr'].append(tmpaddr) #addr

    driver.quit()

    print("Done")
    
    return temp_dict

In [ ]:
# naver_map_search(keyword, category)

coffeeatworks = naver_map_search('커피앳웍스','카페')
bizen = naver_map_search('빚은','떡,한과')
shake = naver_map_search('쉐이크쉑','햄버거')
jamba = naver_map_search('잠바주스','과일,주스전문점')
linas = naver_map_search('리나스','샌드위치')
piginthegarden = naver_map_search('피그인더가든','다이어트,샐러드')
lagrillia = naver_map_search('피그인더가든','이탈리아음식')

# 지방행정인허가데이터

In [ ]:
def Click_Xpath(xpath):
    category = driver.find_element_by_xpath(xpath)
    category.click()

def search(keyword):
    search_box = driver.find_element_by_class_name("input-keyword")
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.ENTER)

# 스크롤 내리기
def ScrollDown():
    # 스크롤 끝까지 내리기
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:   # 0에서부터 영상 맨 마지막까지 긁음.

        # 마우스 스크롤을 끝까지 내리겠음
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        time.sleep(3) 

        # 내린 스크롤까지의 높이를 구해서 변수에 넣음
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height: # 지금의 높이가 웹페이지의 끝이라면
            break # 종료

        last_page_height = new_page_height # 그렇지 않으면 계속 스크롤을 진행

        
# 페이지에서 info찾기
def find_info():
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    names = soup.find_all('td',class_ = "td-upso")
    addrs = soup.find_all('td',class_ = "td-addr")

    tmp = {'name': [], 'addr' : []}
    for name, addr in zip(names, addrs):
        tmp['name'].append(name.text)
        tmp['addr'].append(addr.text)
    
    return tmp

# 페이지 검색 후 데이터 병합까지
def page_search(data):
    tmpData = data.copy()
    
    tmpInfo = find_info()
    tmpData = pd.concat([tmpData, pd.DataFrame(tmpInfo)], axis = 0)
    time.sleep(3)
    
    return tmpData    
    

In [ ]:
url = 'https://www.localdata.kr/data/dataView.do'
data = pd.DataFrame({'name':[], 'addr': []})
keyword = '베이커리팩토리'  # 베이커리 팩토리 

###########################

driver = webdriver.Chrome("C:\\data\\chromedriver_win32\\chromedriver.exe")  #selenium 사용에 필요한 chromedriver.exe 파일 경로 지정
driver.get(url) 

time.sleep(2) #화면 표시 기다리기

# [식품] 선택
Click_Xpath('//*[@id="categoryList"]/ul/li[3]/div[1]')
time.sleep(2)

# [식품제조가공판매] 선택
Click_Xpath('//*[@id="group"]/div[2]/a')
time.sleep(2)

# [제과점영업] 선택
Click_Xpath('//*[@id="opnLeftList"]/li[16]/a/span')
time.sleep(2)

# 검색어 입력
search(keyword)
time.sleep(5)

###########################

ScrollDown()

data = page_search(data)
next_page = True

try : 
    for num in tqdm(range(1,10), desc = 'first pages search...'):
        Click_Xpath(f'//*[@id="navigator"]/a[{num}]')
        ScrollDown()

        data = page_search(data)

    Click_Xpath('//*[@id="navigator"]/a[10]') # 다음 10페이지로 넘어가기
    time.sleep(3)
    
except:
    next_page = False


if next_page == True:
    cnt = 10
    while True:
        try:
            ScrollDown()
            data = page_search(data)

            for num in tqdm(range(3,12), desc = f'{cnt} pages search...'):
                Click_Xpath(f'//*[@id="navigator"]/a[{num}]')
                ScrollDown()

                data = page_search(data)

            Click_Xpath('//*[@id="navigator"]/a[12]')# 다음 10페이지로 넘어가기
            time.sleep(3)

            cnt += 10

        except:
            break


driver.quit()

print('Done')

# Pascucci

In [ ]:
# 파스쿠찌 크롤링하는 코드
page_num = 55
temp = []

for page in tqdm(range(1,page_num+1)):
    list_url ="https://www.caffe-pascucci.co.kr/store/storeList.asp?page=" + str(page)
     
    driver = webdriver.Chrome("C:\\data\\chromedriver_win32\\chromedriver.exe") # 크롬드라이버 불러옴
    driver.implicitly_wait(5) # 10초 기다림
    driver.get(list_url) # 위의 url을 가져옴
    time.sleep(3)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    name = soup.find_all("td", class_ = "storeName")
    address = soup.find_all("p", class_ = "addr")
    
    for n,a in zip(name,address):
        temp.append([n.string,a.string])
    
driver.quit()

print("Done")

# check

In [ ]:
def CheckDuplicate(dataframe):
    checked_df = dataframe.copy()
    checked_df['dup'] = checked_df.duplicated(keep=False)
    
    return checked_df[checked_df.dup == True]